**NAIVE BAYES**

Baseado no teorema de bayes (análise probabilistica)

Aplicações mais comuns :

- Detecção de SPAN
- Detecção de emoções em frases
- Separação de documentos

O naive bayes a partir da tabela de dados previsores gera uma tabela de probabilidades que é usada como base para classificar novos dados.

Vantagens :

- Rápido se comparado a abordagens mais complexas ( Ex: redes neurais )
- Simples
- Capaz de tabalhar com altas dimensões (atributos)
- Boas previsões em bases de dados pequenas ( 400 - 1000 registros)

Desvantagens :
- Presume que os atributos previsores são totalmente independentes, o que nem sempre é verdade.



In [311]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.compose import make_column_transformer


# carregando a base de dados de censo
base = pd.read_csv('../../res/census.csv')
base.shape



(32561, 15)

In [312]:
# separando os dados de previsao e classificacao
previsores = base.iloc[:, 0:14].values
classificadores = base.iloc[:, 14].values

#gerando uma copia dos dados originais para fazer mais testes abaixo
previsores_escalonados=previsores.copy()


#efetuando correcoes nos dados do censo

#transformando dados categorios da base em dados discretos

labelencoder_prev = LabelEncoder()
previsores[:, 1] = labelencoder_prev.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_prev.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_prev.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_prev.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_prev.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_prev.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_prev.fit_transform(previsores[:, 9])
previsores[:, 13] = labelencoder_prev.fit_transform(previsores[:, 13])

preprocess = make_column_transformer(( OneHotEncoder(categories='auto'), [1,3,5,6,7,8,9,13] ),remainder="passthrough") 
previsores = preprocess.fit_transform(previsores).toarray()


In [313]:
# padronizando os valores nao discretos da copia dos previsores ( nao deve ser feito para todos os parametros sob risco de degradar a precisao do algoritimo)

#o min max scaler foi mais interessante para este caso
scaler = MinMaxScaler(feature_range=(0, 200))
#scaler = StandardScaler()

#transformando dados categorios da copia da base em dados discretos
labelencoder_prev = LabelEncoder()
previsores_escalonados[:, 1] = labelencoder_prev.fit_transform(previsores_escalonados[:, 1])
previsores_escalonados[:, 3] = labelencoder_prev.fit_transform(previsores_escalonados[:, 3])
previsores_escalonados[:, 5] = labelencoder_prev.fit_transform(previsores_escalonados[:, 5])
previsores_escalonados[:, 6] = labelencoder_prev.fit_transform(previsores_escalonados[:, 6])
previsores_escalonados[:, 7] = labelencoder_prev.fit_transform(previsores_escalonados[:, 7])
previsores_escalonados[:, 8] = labelencoder_prev.fit_transform(previsores_escalonados[:, 8])
previsores_escalonados[:, 9] = labelencoder_prev.fit_transform(previsores_escalonados[:, 9])
previsores_escalonados[:, 13] = labelencoder_prev.fit_transform(previsores_escalonados[:, 13])


print("\nVisualizando estatisticas dos dados nao discretos antes do escalonamento\n")
for x in range(3):
    print('coluna ',x,"\n")
    print(previsores_escalonados[:,[4,10,12]][x].min())
    print(previsores_escalonados[:,[4,10,12]][x].max())
    print(previsores_escalonados[:,[4,10,12]][x].mean())
    print(previsores_escalonados[:,[4,10,12]][x].var())
    print("\n")


#padronizando dados nao discretos da copia da base original (testes feitos de varias maneiras para daterminar o melhor resultado)
#previsores_escalonados[:,[2,4,10,11,12]] = scaler.fit_transform(previsores_escalonados[:,[2,4,10,11,12]])

#testes com poucas colunas escalonadas
previsores_escalonados[:,[12]] = scaler.fit_transform(previsores_escalonados[:,[12]])
previsores_escalonados[:,[10]] = scaler.fit_transform(previsores_escalonados[:,[10]])
previsores_escalonados[:,[4]] = scaler.fit_transform(previsores_escalonados[:,[4]])

print("\nVisualizando estatisticas dos dados nao discretos depois do escalonamento\n")
for x in range(3):
    print('coluna ',x,"\n")
    print(previsores_escalonados[:,[4,10,12]][x].min())
    print(previsores_escalonados[:,[4,10,12]][x].max())
    print(previsores_escalonados[:,[4,10,12]][x].mean())
    print(previsores_escalonados[:,[4,10,12]][x].var())
    print("\n")


#fazendo o one hot encoder para a copia da base (para os valores discretos)
preprocess = make_column_transformer(( OneHotEncoder(categories='auto'), [1,3,5,6,7,8,9,13] ),remainder="passthrough") 
previsores_escalonados = preprocess.fit_transform(previsores_escalonados).toarray()


#separando os valores de teste e treinamento  para os previsores escalonados e nao escalonados
previsores_treinamento, previsores_teste, classificadores_treinamento1, classificadores_teste1 = train_test_split(previsores, classificadores, test_size=0.15, random_state=0)
previsores_escalonados_treinamento, previsores_escalonados_teste, classificadores_treinamento, classificadores_teste = train_test_split(previsores_escalonados, classificadores, test_size=0.15, random_state=0)




Visualizando estatisticas dos dados nao discretos antes do escalonamento

coluna  0 

13
2174
742.3333333333334
1024956.222222222


coluna  1 

0
13
8.666666666666666
37.55555555555556


coluna  2 

0
40
16.333333333333332
293.5555555555556



Visualizando estatisticas dos dados nao discretos depois do escalonamento

coluna  0 

4.348043480434804
160.0
81.31329340504288
4039.403634595381


coluna  1 

0.0
160.0
61.49659863945578
4951.418390485446


coluna  2 

0.0
106.66666666666667
62.08616780045352
2049.5205187139104




In [314]:
# instanciando o naive bayes com o scikit
classificador = GaussianNB(priors=(.75,.25))
classificador.fit(previsores_escalonados_treinamento, classificadores_treinamento)

# rodando previsoes com o dados de teste (copia)
previsoes_dados_escalonados = classificador.predict(previsores_escalonados_teste)

# fazendo o fit com os dados normais
classificador.fit(previsores_treinamento, classificadores_treinamento1)
previsoes = classificador.predict(previsores_teste)


In [315]:
#testes dessa instancia algoritimo

# o dado de precisao per se nao quer dizer muita coisa e preciso verificar outras metricas
precisao_escalonados = accuracy_score(classificadores_teste, previsoes_dados_escalonados)
precisao = accuracy_score(classificadores_teste1, previsoes)

# uma dessas metricas eh a matriz de confusao ... ela e capaz de mostrar o desempenho do algoritimo para cada classe 
matriz_escalonados = confusion_matrix(classificadores_teste, previsoes_dados_escalonados)
matriz = confusion_matrix(classificadores_teste1, previsoes)

#o scikit tambem possui uma classe utilitaria que prove um report mais detalhado...
report_escalonados = classification_report(classificadores_teste, previsoes_dados_escalonados)
report = classification_report(classificadores_teste1, previsoes)

print("Precisão dados normais / escalonados :\n")
print(precisao,'/',precisao_escalonados)
print("\nMatriz de confusão dados normais / escalonados:\n")
print(matriz)
print("\n")
print(matriz_escalonados)
print("\nReport dados normais / escalonados:\n")
print (report)
print("\n")
print (report_escalonados)

Precisão dados normais / escalonados :

0.7950870010235415 / 0.7985670419651996

Matriz de confusão dados normais / escalonados:

[[3515  178]
 [ 823  369]]


[[3565  128]
 [ 856  336]]

Report dados normais / escalonados:

              precision    recall  f1-score   support

       <=50K       0.81      0.95      0.88      3693
        >50K       0.67      0.31      0.42      1192

    accuracy                           0.80      4885
   macro avg       0.74      0.63      0.65      4885
weighted avg       0.78      0.80      0.77      4885



              precision    recall  f1-score   support

       <=50K       0.81      0.97      0.88      3693
        >50K       0.72      0.28      0.41      1192

    accuracy                           0.80      4885
   macro avg       0.77      0.62      0.64      4885
weighted avg       0.79      0.80      0.76      4885



<br>**TODO :Verificar a base de treinamento para melhorar a distribuicao das classes e verificar se ha alguma melhora**